<div style="background-color:AliceBlue; padding:10px; border-left:5px solid #6495ed; margin-bottom:10px;">
  <h4 style="font-size:13px;">📦 <strong>Importar Librerias</strong></h4>
</div>

In [1]:
import pandas as pd
import sys, ray
from sklearn.datasets import load_iris
import requests

<div style="background-color:AliceBlue; padding:10px; border-left:5px solid #6495ed; margin-bottom:10px;">
  <h4 style="font-size:13px;">📦 <strong>Init ray server</strong></h4>
</div>

In [2]:
try:
    ray.init(address="ray://127.0.0.1:10001")
    print("✅ Conectado a Ray existente.")
except Exception as e:
    sys.exit(f"❌ No se pudo conectar a ray://127.0.0.1:10001 → {e}")

2025-08-10 19:20:37,281	INFO client_builder.py:242 -- Passing the following kwargs to ray.init() on the server: log_to_driver
SIGTERM handler is not set because current thread is not the main thread.


✅ Conectado a Ray existente.


<div style="background-color:AliceBlue; padding:10px; border-left:5px solid #2e8b57; margin-bottom:10px;">
  <h4 style="font-size:13px;">🤖 <strong>Verifica que Modelos estan Desplegados</strong></h4>
</div>

In [3]:
iris = load_iris()
df = pd.DataFrame(iris.data, columns=iris.feature_names)
df["target"] = iris.target
df = df.drop("target", axis=1)

df = df.rename(
    columns = {
        'sepal length (cm)': 'sepal_length',
        'sepal width (cm)' : 'sepal_width',
        'petal length (cm)': 'petal_length',
        'petal width (cm)' : 'petal_width'
    }
)

df_sample = df.sample( n = 5 )
df_request = df_sample.to_dict('records')

In [4]:
df_request

[{'sepal_length': 6.1,
  'sepal_width': 2.8,
  'petal_length': 4.7,
  'petal_width': 1.2},
 {'sepal_length': 5.5,
  'sepal_width': 2.4,
  'petal_length': 3.8,
  'petal_width': 1.1},
 {'sepal_length': 5.2,
  'sepal_width': 2.7,
  'petal_length': 3.9,
  'petal_width': 1.4},
 {'sepal_length': 5.8,
  'sepal_width': 4.0,
  'petal_length': 1.2,
  'petal_width': 0.2},
 {'sepal_length': 5.5,
  'sepal_width': 2.6,
  'petal_length': 4.4,
  'petal_width': 1.2}]

<div style="background-color:AliceBlue; padding:10px; border-left:5px solid #2e8b57; margin-bottom:10px;">
  <h4 style="font-size:13px;">🤖 <strong>Verifica que Modelos estan Desplegados</strong></h4>
</div>

In [5]:
try:
    for i in range(len(df_sample)):
        response = requests.post(
            "http://127.0.0.1:8000/v1/iris-logreg-modelo/score/",
            json = df_request[i]
        )
        result = response.json()
        print(df_request[i], "--> ", result)
        print("-" * 120)
        
except Exception as ex:
    print(f"Request: Se produjo el siguiente error : {ex} ")

{'sepal_length': 6.1, 'sepal_width': 2.8, 'petal_length': 4.7, 'petal_width': 1.2} -->  {'cod': 200, 'msg': 'Score calculated successfully.', 'data': {'em_eventprobability': 0.8181885135384775}, 'errors': None}
------------------------------------------------------------------------------------------------------------------------
{'sepal_length': 5.5, 'sepal_width': 2.4, 'petal_length': 3.8, 'petal_width': 1.1} -->  {'cod': 200, 'msg': 'Score calculated successfully.', 'data': {'em_eventprobability': 0.9467412567681603}, 'errors': None}
------------------------------------------------------------------------------------------------------------------------
{'sepal_length': 5.2, 'sepal_width': 2.7, 'petal_length': 3.9, 'petal_width': 1.4} -->  {'cod': 200, 'msg': 'Score calculated successfully.', 'data': {'em_eventprobability': 0.8997109568085427}, 'errors': None}
------------------------------------------------------------------------------------------------------------------------
{'se